In [ ]:
import numpy as np
import pyci
import psi4

In [ ]:
# ! s-type d-aug-cc-pVQZ functions
# ! s-type ARO functions
# ! p-type d-aug-cc-pVQZ functions
# ! p-type ARO functions
# ! d-type d-aug-cc-pVQZ functions
# ! d-type ARO functions
# ! f-type d-aug-cc-pVQZ functions
# ! f-type ARO functions
basis_dict = {
      'basstring': """
****
He     0
S    7   1.00
    528.5000000              0.0009400
     79.3100000              0.0072140
     18.0500000              0.0359750
      5.0850000              0.1277820
      1.6090000              0.3084700
      0.5363000              0.4530520
      0.1833000              0.2388840
S    1   1.00
      1.6090000              1.0000000
S    1   1.00
      0.5363000              1.0000000
S    1   1.00
      0.1833000              1.0000000
S    1   1.00
      0.0481900              1.0000000
S    1   1.00
      0.0127                 1.0000000
S    1   1.00
      4.142630e-01           1.0000000
S    1   1.00
      8.204444e-02           1.0000000
S    1   1.00
      2.244417e-02           1.0000000
S    1   1.00
      6.908733e-03           1.0000000
S    1   1.00
      2.252960e-03           1.0000000
S    1   1.00
      6.972017e-04           1.0000000
P    1   1.00
      5.9940000              1.0000000
P    1   1.00
      1.7450000              1.0000000
P    1   1.00
      0.5600000              1.0000000
P    1   1.00
      0.1626000              1.0000000
P    1   1.00
      0.0472                 1.0000000
P    1   1.00
      4.442081e-01           1.0000000
P    1   1.00
      9.422795e-02           1.0000000
P    1   1.00
      2.719846e-02           1.0000000
P    1   1.00
      8.859505e-03           1.0000000
P    1   1.00
      3.080030e-03           1.0000000
P    1   1.00
      1.066130e-03           1.0000000
D    1   1.00
      4.2990000              1.0000000
D    1   1.00
      1.2230000              1.0000000
D    1   1.00
      0.3510000              1.0000000
D    1   1.00
      0.101                  1.0000000
D    1   1.00
      3.348153e-01           1.0000000
D    1   1.00
      8.135966e-02           1.0000000
D    1   1.00
      2.631020e-02           1.0000000
D    1   1.00
      9.484616e-03           1.0000000
D    1   1.00
      3.635094e-03           1.0000000
D    1   1.00
      1.407186e-03           1.0000000
F    1   1.00
      2.6800000              1.0000000
F    1   1.00
      0.6906000              1.0000000
F    1   1.00
      0.178                  1.0000000
F    1   1.00
      3.463980e-01           1.0000000
F    1   1.00
      8.331040e-02           1.0000000
F    1   1.00
      2.805333e-02           1.0000000
F    1   1.00
      1.056283e-02           1.0000000
F    1   1.00
      4.235223e-03           1.0000000
F    1   1.00
      1.732138e-03           1.0000000
****
""",
}

In [ ]:
# Lets intiate AOint() a subclass of psi4utils()
basis = 'd-aug-cc-pvqz'
molfile = '../.xyz/He.xyz'
mol = pyci.utils.molecule(basis, molfile, wd='./', 
                 psi4mem='2 Gb', numpymem=2, 
                 custom_basis=False, basis_dict=None,
                 store_wfn=False, properties=['dipoles']) 


In [ ]:
cisd = pyci.configint.CISD(mol,{'doubles':False})

In [ ]:
HCISD = cisd.HCISD
HCISD0 = HCISD - mol.scf_energy*np.eye(sum(cisd.num_csfs))
cisd_pyci = cisd.energy()
if cisd.options['doubles']:
    cisd_psi4 = psi4.energy('CISD')
else:
    cisd_psi4 = psi4.energy('scf')
print("pyci CISD E0: {e:16.16f}".format(e=cisd_pyci))
print("psi4 CISD E0: {e:16.16f}".format(e=cisd_psi4))
print("dE : {dE:1.2E}".format(dE=abs(cisd_psi4- cisd_pyci)))

In [ ]:
csf_dpx, csf_dpy, csf_dpz = cisd.get_dipoles()

In [ ]:
vals, vecs = cisd.get_eigen()

In [ ]:
from pyci.tdci import excite
fs_to_au = 41.341374575751
ti = 0.0 
tf = 20.0*fs_to_au
dt = 0.0001*fs_to_au 
sigma = 5.1*fs_to_au
E0 = 0.07549
w0 = 0.056
phase = 0.0
psi0 = vecs[0]
time_params =  ti, tf, dt

In [9]:
# Ruge Kutta
from pyci.integrators import RK4
laser_params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t :  -1j * (HCISD0 - excite.sin2_pulse(t, laser_params)*csf_dpz)
propagator = RK4(func, psi0, time_params)
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], 
                            ops_headers=['dpx', 'dpy', 'dpz', 'energy'], 
                            print_nstep= 100, outfile='rk4_tdprop.txt')

In [ ]:
# Split Operator
from pyci.integrators.splitoperator import SplitOperator 
laser_params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t : (excite.sin2_pulse(t, laser_params)*csf_dpz)
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = SplitOperator(vals, vecs, func, psi0, time_params)
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], 
                            ops_headers=['dpx', 'dpy', 'dpz', 'energy'], 
                            print_nstep= 100, outfile='so_tdprop.txt')

In [ ]:
# Heuristic Lifetime model
from pyci.tdci.lifetime import heuristic
model = heuristic(vals, vecs, cisd.csfs, mol.mo_eps[0])
IP = 0.90357185430291 # 24.587387 eV
ip_params = E0, w0, IP
cmplx_vals = model.cmplx_energies(ip_params, use_d2=False)


In [ ]:
# Split Operator with lifetimes
from pyci.integrators.splitoperator import SplitOperator 
func = lambda t : excite.sin2_pulse(t, laser_params)*csf_dpz
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = SplitOperator(cmplx_vals, vecs, func, psi0, time_params)
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], 
                            ops_headers=['dpx', 'dpy', 'dpz', 'energy'], 
                            print_nstep= 100, outfile='lt_so_tdprop.txt')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
rk4_data = np.loadtxt('rk4_tdprop.txt',skiprows=1)
so_data = np.loadtxt('so_tdprop.txt',skiprows=1)
lt_so_data = np.loadtxt('lt_so_tdprop.txt',skiprows=1)
plt.figure(figsize=(10,8), facecolor='white')
plt.plot(rk4_data[:,0],rk4_data[:,4], '--', label='RK4 w/o lifetime')
plt.plot(so_data[:,0],so_data[:,4], '--', label='SO w/o lifetime')
plt.plot(lt_so_data[:,0],lt_so_data[:,4], '-.',label='w/ lifetime')
plt.legend()

In [ ]:
i = 5
plt.figure(figsize=(10,8), facecolor='white')
plt.plot(rk4_data[:,0], rk4_data[:,i]-rk4_data[0,i], '--', label='RK4 w/o lifetime')
plt.plot(so_data[:,0], so_data[:,i]-so_data[0,i], '--', label='SO w/o lifetime')
plt.plot(lt_so_data[:,0], lt_so_data[:,i]-lt_so_data[0,i], '-.',label='w/ lifetime')
plt.legend()

In [ ]:
from pyci.tdci import analysis
def get_hhg_data(filename):
    time_fs, dipoles, energy, norm = analysis.get_prop_data(filename)
    dpz = dipoles[2]
    dpz_vel, dpz_acc = analysis.calc_moments(time_fs,dpz)
    freq, G_obs, G_obs_vel, G_obs_acc = analysis.calc_Gobs(time_fs, dpz, 
                                                return_moments=True, 
                                                obs_vel=dpz_vel, 
                                                obs_acc=dpz_acc, dt_fs=1e-4)
    return freq, G_obs, G_obs_vel, G_obs_acc     

fig, axes = plt.subplots(3,2, facecolor='white',figsize=(12,10))
freq_so, G_obs_so, G_obs_vel_so, G_obs_acc_so = get_hhg_data('so_tdprop.txt')
freq_lt_so, G_obs_lt_so, G_obs_vel_lt_so, G_obs_acc_lt_so = get_hhg_data('lt_so_tdprop.txt')
analysis.plot_hhg(freq_so, G_obs_so, G_obs_vel_so, G_obs_acc_so, axes[:,0], xmax=25, label='w/o lifetimes')
analysis.plot_hhg(freq_lt_so, G_obs_lt_so, G_obs_vel_lt_so, G_obs_acc_lt_so, axes[:,1],xmax=25, label='w/ lifetimes')